In [5]:
from googleapiclient.discovery import build
import pandas as pd


In [6]:
api_key = ''
channel_id = 'UC_i8X3p8oZNaik8X513Zn1Q'

youtube = build('youtube', 'v3', developerKey=api_key)

### Fonction pour récupérer les infos sur la chaîne YT

In [7]:
def get_channel_stats(youtube, channel_id):
     request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
     response = request.execute()

     data = dict(Channel_name = response['items'][0]['snippet']['title'],
                 Subscribers = response['items'][0]['statistics']['subscriberCount'],
                 Total_videos = response['items'][0]['statistics']['videoCount'],
                 playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
     
     return data


In [8]:
channel_statistics = get_channel_stats(youtube, channel_id)

In [9]:
channel_data = pd.DataFrame([channel_statistics], index=['channel_statistics'])

channel_data

,Channel_name,Subscribers,Total_videos,playlist_id
channel_statistics,FilmsActu,4580000,11737,UU_i8X3p8oZNaik8X513Zn1Q


In [10]:
playlist_id = channel_data.loc[channel_data['Channel_name']=='FilmsActu', 'playlist_id'].iloc[0]
playlist_id

'UU_i8X3p8oZNaik8X513Zn1Q'

In [11]:
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId= playlist_id,
        maxResults = 50
    
    )
    response = request.execute()

    video_ids = []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId= playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids


    # more_pages = True

    # while more_pages:
    #     if next_page_token is None:
    #         more_pages = False

    #     else:
    #         request = youtube.playlistItems().list(
    #     part="contentDetails",
    #     playlistId= playlist_id,
    #     maxResults = 50,
    #     pageToken = next_page_token
    #     )
            
    # response = request.execute()

    # for i in range(len(response['items'])):
    #     video_ids.append(response['items'][i]['contentDetails']['videoId']) 

    # next_page_token = response.get('nextPageToken')

    # return len(video_ids)


In [12]:
video_ids = get_video_ids(youtube, playlist_id)
video_ids

['Y8b0Xq5qTnE',
 'urPKr_WBxug',
 '-NqbKnFAGWE',
 'nkzFemum490',
 'vwq0oteygFM',
 'DT4prTV1jF8',
 'nU6YqPFz044',
 'MMlW6gZEe90',
 'Yze-9ANhIyA',
 'FLY-b62RLyA',
 'x2qwnQRU1b8',
 'uWokscU2Hnk',
 'sRhLHCI-7MY',
 'DBAf34NgY-0',
 'Y7GX3iPk0xE',
 'mv8ZR-Z_jkw',
 'rMLHMAKcyrg',
 'tXvQH0xJdDQ',
 'cprG6K7_n_k',
 'oEt3jxvefvo',
 '85oldjE5qy0',
 'bDiJ1FR86Dc',
 'h4RGUn5AdaU',
 'RmF0cxZcpIE',
 'fq6qFr0zVlw',
 'PWRxewJHuVU',
 'TvOIhl1p18M',
 'sw6bnTQ37Fo',
 'ilU15hwe3-0',
 'fONkV6TQmrE',
 'l4jyaC7GR0A',
 'smA8whgjLD8',
 'nde6DOMOSgU',
 'SdWzp3Vn0X0',
 'Fz32MNJrWbc',
 'DpnRyFo7Wb0',
 'XznXYvHo6-I',
 'vJy5c2aU7Rw',
 'gCb4RWoILQ8',
 'w3uXWs6RsrE',
 'ibry8-ezkco',
 'yp-xhI7FETY',
 '77jLbZt3AMs',
 '6on41bCmXdU',
 '2d7JG8yDMUY',
 '3tP1FiIU-mQ',
 'vGcXphgn02c',
 'WRoOM8SGORM',
 'a-eG0Ays8s8',
 '-XAwrqnSL_E',
 'qYdMBzOvf2Q',
 '7K0Da3nh17E',
 '2LD-3ldZ_Ow',
 'QYhOhjxJqdg',
 'Yrhn1nwdzQk',
 'QqlPT2sKneU',
 'WWqHyM7DsBI',
 'zwZCLmEZP14',
 'wHxszab3BSE',
 'kPhxXPthHzg',
 'JeRg2Uo5BpA',
 'PUt03rJzNxE',
 'Gn8Qkm

### Fonction pour prendre les détails des vidéos

In [13]:
def get_video_details(youtube, video_ids):
    all_video_stats =[]

    for i in range (0, len(video_ids), 50):

        request = youtube.videos().list(
            part="snippet,statistics",
            id=','.join(video_ids[i:i+50])
        )
        
        response = request.execute()

        for video in response['items']: 
            video_stats = dict(Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               View_count = video['statistics']['viewCount'],
                               Like_count = video['statistics']['likeCount'],
                               Comment_count = video['statistics'].get('commentCount', 'N/A'),
                               Tags = video['snippet'].get('tags', 'N/A'),
                               )
            all_video_stats.append(video_stats)

    return all_video_stats
    


In [14]:
video_details = get_video_details(youtube, video_ids)

df = pd.json_normalize(video_details)

df.to_csv('video_details.csv', index=False)